In [1]:
import findspark
findspark.init('/home/fede/spark-2.1.0-bin-hadoop2.7')

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('rec').getOrCreate()

In [3]:
from pyspark.ml.recommendation import ALS

In [4]:
from pyspark.ml.evaluation import RegressionEvaluator

In [5]:
file_path = '/home/fede/movielens_ratings.csv'
data = spark.read.csv(file_path, inferSchema=True, header=True)

In [6]:
data.show()

+-------+------+------+
|movieId|rating|userId|
+-------+------+------+
|      2|   3.0|     0|
|      3|   1.0|     0|
|      5|   2.0|     0|
|      9|   4.0|     0|
|     11|   1.0|     0|
|     12|   2.0|     0|
|     15|   1.0|     0|
|     17|   1.0|     0|
|     19|   1.0|     0|
|     21|   1.0|     0|
|     23|   1.0|     0|
|     26|   3.0|     0|
|     27|   1.0|     0|
|     28|   1.0|     0|
|     29|   1.0|     0|
|     30|   1.0|     0|
|     31|   1.0|     0|
|     34|   1.0|     0|
|     37|   1.0|     0|
|     41|   2.0|     0|
+-------+------+------+
only showing top 20 rows



In [8]:
data.describe().show()

+-------+------------------+------------------+------------------+
|summary|           movieId|            rating|            userId|
+-------+------------------+------------------+------------------+
|  count|              1501|              1501|              1501|
|   mean| 49.40572951365756|1.7741505662891406|14.383744170552964|
| stddev|28.937034065088994| 1.187276166124803| 8.591040424293272|
|    min|                 0|               1.0|                 0|
|    max|                99|               5.0|                29|
+-------+------------------+------------------+------------------+



In [9]:
training,test = data.randomSplit([0.8,0.2])

In [10]:
als = ALS(maxIter=5,regParam=0.01,userCol='userId',itemCol='movieId',
          ratingCol='rating')

In [11]:
model = als.fit(training)

In [12]:
predictions = model.transform(test)

In [13]:
predictions.show()

+-------+------+------+-----------+
|movieId|rating|userId| prediction|
+-------+------+------+-----------+
|     31|   1.0|    13|  1.1582725|
|     31|   1.0|    29|  1.6768869|
|     31|   1.0|    18|  3.2552536|
|     85|   1.0|    25|  2.7400243|
|     85|   1.0|    29| 0.20920515|
|     85|   3.0|    21|  2.4446278|
|     65|   1.0|    16|   2.203344|
|     65|   2.0|     3| -2.2060463|
|     65|   1.0|     2|-0.30666932|
|     78|   1.0|    27|   1.018314|
|     34|   1.0|    17| -4.3884544|
|     81|   5.0|    28|  0.8425004|
|     81|   1.0|    15|  1.0973904|
|     81|   1.0|     7|  2.0311768|
|     28|   1.0|     5|  2.3385417|
|     28|   1.0|     2| -2.3515162|
|     28|   5.0|    18|  1.8073947|
|     76|   1.0|    26|  2.5265574|
|     76|   1.0|     1|  3.3213725|
|     76|   1.0|    20|  3.0229254|
+-------+------+------+-----------+
only showing top 20 rows



In [14]:
evaluator = RegressionEvaluator(metricName='rmse',labelCol='rating',
                               predictionCol='prediction')

In [15]:
rmse = evaluator.evaluate(predictions)

In [16]:
print('RMSE')
print(rmse)

RMSE
2.095252747788362


In [17]:
single_user = test.filter(test['userId']==11).select(['movieId','userId'])

In [18]:
single_user.show()

+-------+------+
|movieId|userId|
+-------+------+
|     11|    11|
|     13|    11|
|     20|    11|
|     22|    11|
|     25|    11|
|     30|    11|
|     38|    11|
|     59|    11|
|     64|    11|
|     75|    11|
|     80|    11|
|     86|    11|
+-------+------+



In [19]:
recommendations = model.transform(single_user)

In [20]:
recommendations.orderBy('prediction',ascending=False).show()

+-------+------+----------+
|movieId|userId|prediction|
+-------+------+----------+
|     25|    11| 4.0451336|
|     75|    11| 3.5528188|
|     22|    11| 3.0281332|
|     80|    11| 2.8409507|
|     38|    11|  2.608005|
|     13|    11| 2.6046467|
|     64|    11| 1.9941509|
|     86|    11| 1.0942696|
|     30|    11|0.88414276|
|     59|    11|0.60464376|
|     20|    11|0.18778116|
|     11|    11|-0.7285641|
+-------+------+----------+

